In [ ]:
from Data_Prep import Data_Prep, Player_IO
from sklearn.model_selection import train_test_split # type: ignore
import torch
import Prep_Map
import Output_Map
from torch.optim import lr_scheduler
from Player_Dataset import Player_Dataset, Create_Test_Train_Datasets
import Model_Train
from Player_Model import RNN_Model, LayerArch
from tqdm.notebook import tqdm
from Constants import device
import seaborn as sns
from pylab import savefig
from matplotlib import pyplot as plt
import sys
from Constants import DEFAULT_HIDDEN_SIZE_HITTER, DEFAULT_NUM_LAYERS_HITTER
import numpy as np

    
filename = "ClassArchLargePeak"
output_type = "Class"
batch_size = 600
ys = range(5, 8)
xs = range(70, 171, 10)
data = []
title = "Class Loss vs Ouput Layer Architecture"

x_label = "Num Layers"
y_label = "Layer Size"

data_prep = Data_Prep(Prep_Map.base_prep_map, Output_Map.base_output_map)
hitter_io_list = data_prep.Generate_IO_Hitters("WHERE lastMLBSeason<? AND signingYear<? AND isHitter=?", (2025,2015,1), use_cutoff=True)
train_dataset, test_dataset = Create_Test_Train_Datasets(hitter_io_list, 0.25, 0)

num_layers = DEFAULT_NUM_LAYERS_HITTER
hidden_size = DEFAULT_HIDDEN_SIZE_HITTER
hitting_mutators = data_prep.Generate_Hitting_Mutators(batch_size, Player_IO.GetMaxLength(hitter_io_list))

stats_arch = LayerArch(layer_size=50, num_layers=2)

with tqdm(total=len(xs) * len(ys), desc="Total Training Runs") as pbar:
    for num_layers in ys:
        z = []
        for layer_size in xs:
            warclass_arch = LayerArch(layer_size=layer_size, num_layers=num_layers)
            network = RNN_Model(train_dataset.get_input_size(), 
                                num_layers, 
                                hidden_size, 
                                hitting_mutators, 
                                data_prep=data_prep, 
                                is_hitter=True, 
                                stats_arch=stats_arch,
                                warclass_arch=warclass_arch)
            network = network.to(device)

            training_generator = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            testing_generator = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
            
            
            best_loss = Model_Train.trainAndGraph(network, 
                                                    train_dataset,
                                                    test_dataset,
                                                    batch_size=batch_size,
                                                    num_epochs=41, 
                                                    logging_interval=10000, 
                                                    early_stopping_cutoff=40, 
                                                    should_output=False, 
                                                    model_name=f"Models/exp",
                                                    save_last=False,
                                                    elements_to_save=[0])
            z.append(best_loss[0])
            pbar.update(1)
        data.append(z)
        
        


In [ ]:
plt.figure(figsize=(1 * len(xs), .75 * len(ys) + 2))
heatmap = sns.heatmap(data, xticklabels=xs, yticklabels=ys, annot=True, fmt=".3f")
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(title)
plt.tight_layout(pad=0.25)
plt.savefig(f'Experiments/{filename}_{output_type}.png', dpi=400)